**RTSA 2023**


# Project C: Non-LTE Line Formation

This project is based on exercises by Jorrit Leenaarts (Stockholm University).

#### Header and imports

In [1]:
%matplotlib inline

import numpy

from scipy.special import wofz   # for Voigt function
from scipy.special import roots_legendre  # for Gaussian quadrature
from scipy.sparse import diags
from scipy.linalg import solve_banded

from numba import njit

import matplotlib.pyplot as plt

## 1. Background

This project will lead you through the basics of non-LTE radiative transfer. Compared to the other two projects it is more abstract and we will not compute line profiles as detailed as in Project B, since this would require a lot of boilerplate code. The focus is instead on the numerical techniques that make solving the problem possible, and you will get to design a non-LTE scheme from scratch.

### 1.1 The Feautrier method

The Feautrier method is a way to compute intensity for a given source function and optical depth scale. It can be used for both LTE and non-LTE problems, and it approaches the radiative transport equation as a boundary-value problem. It has some limitations (e.g. cannot be used with line blends, needs additional treatment for velocities), and when used in 3D it works better for long-characteristics. For these reasons, it is often favoured to use piecewise integration methods instead. However, the Feautrier method is still particularly illustrative to interpret the $\Lambda$ operator, so we will use it here.

We start by following the lecture notes (Sect 5.2) and define:

$$
\begin{aligned}
P(\tau, \mu) &= \frac{1}{2}\left[I^+(\tau, \mu) + I^-(\tau, \mu) \right],\\
R(\tau, \mu) &= \frac{1}{2}\left[I^+(\tau, \mu) - I^-(\tau, \mu) \right],
\end{aligned}
$$
so we can write the transport equation as a second order equation for $P$:

\begin{equation}
\frac{d^2 P(\tau, \mu)}{d\tau^2} = P(\tau, \mu) - S(\tau).
\end{equation}

The angle-averaged intensity $J$ can be obtained by summing over $P$ for outgoing rays:

\begin{equation}
J(\tau) = \int_0^1 P(\tau, \mu) d\mu.
\end{equation}

Writing Eq. (1) as a difference equation, we obtain a system of equations of the form:

$$
-A_i P_{i-1} + B_i P_i - C_i P_{i+1} = S_i,
$$
which in matrix form can be written as:
$$
\mathbf{T}P = S,
$$
Where $\mathbf{T}$ is a $n\times n$ tridiagonal matrix, where $n$ is the number of depth points in the atmosphere, $-A_i$ are the subdiagonal elements, $B_i$ the diagonal elements, and $-C_i$ the supradiagonal elements. This system can be solved by Gaussian elimination, but it is much faster to use algorithms for tridiagonal systems, which make use of forward-backward substitution. The values for $A_i$, $B_i$, and $C_i$ are given in the lecture notes (5.23-5.25), but using our notation we set $\mu=1$ and instead change $\tau\to \tau/\mu$ for different angles.

This function can be used to compute the $\mathbf{T}$ matrix:

In [6]:
def Tmatrix(τ, μ, format=None):
    tau = τ / μ
    n = len(tau)
    Δτ = numpy.roll(tau, -1) - tau
    Δτm = numpy.roll(Δτ, 1)

    A = 2 / (Δτm * (Δτm + Δτ))
    B = 1 + 2 / (Δτ * Δτm)
    C = 2 / (Δτ * (Δτm + Δτ))

    # Boundary conditions top
    B[0] = 2 / Δτ[0]**2 + 2 / Δτ[0] + 1
    C[0] = 2 / Δτ[0]**2
    # Boundary conditions bottom
    A[n-1] = 2 / (Δτ[n-2] * (Δτ[n-2] + 2))
    B[n-1] = (2 + 2*Δτ[n-2] + Δτ[n-2]**2) / (Δτ[n-2] * (Δτ[n-2] + 2))

    if format == 'banded':  # for use with scipy.linalg.solve_banded
        T = numpy.zeros((3, n))
        T[0, 1:] = -C[:-1]
        T[1] = B
        T[2, :-1] = -A[1:]
        return T
    elif format == 'sparse':  # for use with scipy.sparse.linalg.spsolve
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1], format='csc')
    else:  # for use with numpy.linalg.solv
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1]).toarray()

Note that `Tmatrix` by default will return a 2D array, but if you use `format='banded'`, it will output an array of shape `(3, N)` with only the three diagonals, in a format that can be used with `scipy.linalg.solve_banded`, which allows for great speed gains.

$J$ can be obtained by summing over the $P$ for different directions, typical following an angle quadrature:
$$
J(\tau) = \int_0^1 P(\tau, \mu) d\mu \approx \sum_i w_i P(\tau, \mu_i)
$$

In 1D problems, we typically use a Gaussian quadrature with $k$ points, scaled to the interval of $[0, 1]$, which is obtained from Legendre polynomials:

In [ ]:
def quadrature(k=5):
    """
    Returns the nodes and weights form a Gaussian
    quadrature with k points. Rescaled to an interval
    from [0, 1].
    """
    nodes, weights = roots_legendre(k)
    return nodes/2 + 0.5, weights/2

We can also write $J$ in terms of the $\Lambda$ operator, which operates on $S$:

$$
J = \mathbf{\Lambda}S,
$$
and therefore, for an angle quadrature, 
$$
J = \sum_i\mathbf{T}^{-1}(\mu_i)S,
$$

In the last exercise of this project, you can use the following optimised Feautrier solver, based on an algorithm by Rybicki and Hummer (1991):

In [7]:
@njit
def feautrier(tau, S, doLstar=False, doI=False):
    """
    Feautrier formal solver taken from the RH code (Uitenbroek 2001),
    adapted from version by Jorrit Leenaarts.

    Evaluate sum of monochromatic intensities (I^+ + I^-)/2 and
    returns emergent intensity I_surface along a ray with given
    optical depth scale tau and source function S.

    Numerical scheme formulated as in Appendix A of:
    G.B. Rybicki & D.G Hummer, A&A 245, 171-181.
    
    Parameters
    ----------
    tau : 1D array
        Array with optical depths. Must be divided by μ for 
        incline rays.
    S : 1D array
        Source function.
        
    Returns
    -------
    P : 1D array
        Feautrier P, defined as 1/2 * (I^+ - I^-)
    Psi : 1D array
        L^*, diagonal of Λ operator. Only if doLstar=True.
    iplus : 1D array
        I^+, outgoing intensity. Only if doI=True.
    iminus : 1D array
        I^+, ingoing intensity. Only if doI=True.
    """
    nz = len(tau)

    Psi = None
    iplus = None
    iminus = None

    dtau = numpy.zeros_like(tau)
    abc = numpy.zeros_like(tau)
    A1 = numpy.zeros_like(tau)
    C1 = numpy.zeros_like(tau)
    F = numpy.zeros_like(tau)
    ztmp = numpy.zeros_like(tau)
    Stmp = numpy.zeros_like(tau)
    P = numpy.zeros_like(tau)

    for k in range(nz-1):
        dtau[k] = tau[k+1]-tau[k]

    # upper boundary is zero incoming radiation, I(-) = R0*I(+) + H0
    r0 = 0.0
    h0 = 0.0

    f0 = (1.0 - r0) / (1.0 + r0)
    abc[0] = 1.0 + 2.0 * f0 / dtau[0]
    C1[0] = 2.0 / dtau[0]**2
    Stmp[0] = S[0] + 2.0 * h0 / ((1.0 + r0) * dtau[0])

    # lower boundary is thermalized, I(+) = Rn*I(-) + Hn
    rN = 0.0
    hN = S[nz-1] + (S[nz-1] - S[nz-2]) / dtau[nz-2]

    fN = (1.0 - rN) / (1.0 + rN)
    abc[nz-1] = 1.0 + 2.0 * fN / dtau[nz-2]
    A1[nz-1] = 2.0 / dtau[nz-2]**2
    Stmp[nz-1] = S[nz-1] + 2.0*hN / ((1.0 + rN) * dtau[nz-2])

    for k in range(1, nz-1):
        dtau_mid = 0.5 * (dtau[k] + dtau[k-1])
        A1[k] = 1.0 / (dtau_mid * dtau[k-1])
        C1[k] = 1.0 / (dtau_mid * dtau[k])
        abc[k] = 1.0
        Stmp[k] = S[k]

    # Start the elimination
    F[0] = abc[0] / C1[0]
    ztmp[0] = Stmp[0] / (abc[0] + C1[0])
    for k in range(1, nz-1):
        F[k] = (abc[k] + A1[k]*F[k-1]/(1.0 + F[k-1])) / C1[k]
        ztmp[k] = (Stmp[k] + A1[k]*ztmp[k-1]) / (C1[k] * (1.0 + F[k]))

    # backsubstitution
    P[nz-1] = ((Stmp[nz-1] + A1[nz-1]*ztmp[nz-2]) /
               (abc[nz-1] + A1[nz-1]*(F[nz-2] / (1.0 + F[nz-2]))))
    for k in range(nz-2, -1, -1):
        P[k] = P[k+1] / (1.0 + F[k]) + ztmp[k]

    if (doLstar):
        Psi = numpy.zeros_like(tau)
        G = numpy.zeros_like(tau)
        # diagononal operator
        G[nz-1] = abc[nz-1] / A1[nz-1]

        for k in range(nz-2, 0, -1):
            G[k] = (abc[k] + C1[k]*G[k+1]/(1.0 + G[k+1])) / A1[k]

        Psi[0] = 1.0 / (abc[0] + C1[0]*G[1]/(1.0 + G[1]))

        for k in range(1, nz-1):
            Psi[k] = 1.0 / (abc[k] + A1[k]*F[k-1]/(1.0 + F[k-1]) +
                            C1[k]*G[k+1]/(1.0 + G[k+1]))

        Psi[nz-1] = 1.0 / (abc[nz-1] + A1[nz-1]*F[nz-2]/(1.0 + F[nz-2]))

    if (doI):
        Q = numpy.zeros_like(tau)
        qmid = numpy.zeros_like(tau)
        PmS = numpy.zeros_like(tau)
        #  compute q and then I+ and I-
        PmS[0] = P[0] - S[0]
        Q[0] = (P[0] * (1.0 - r0) - h0)/(1.0 + r0)
        qmid = numpy.copy(Q)
        qmid[0] += 0.5 * dtau[0] * PmS[0]

        for k in range(1, nz-1):
            if (abs(A1[k]) > 1.0):
                PmS[k] = P[k] - S[k]
            else:
                PmS[k] = C1[k]*(P[k+1] - P[k]) - A1[k]*(P[k] - P[k-1])
            qmid[k] = qmid[k-1] + 0.5 * (dtau[k] + dtau[k-1]) * PmS[k]

        for k in range(1, nz-1):
            Q[k] = (dtau[k] * qmid[k-1] + dtau[k-1]
                    * qmid[k]) / (dtau[k] + dtau[k-1])

        PmS[nz-1] = P[nz-1] - S[nz-1]
        Q[nz-1] = -((1.0 - rN) / (1.0 + rN)) * P[nz-1] + hN/(1.0 + r0)
        iplus = P + Q
        iminus = P - Q

    return P, Psi, iplus, iminus

### 1.2 Non-LTE problems with coherent scattering for a two-level atom

In the case of purely coherent scattering we can express the source function as a scattering term and a thermal term, using the photon destruction probability $\varepsilon$:

$$
S = (1 - \varepsilon)J + \varepsilon B,
$$
or, with the $\mathbf{\Lambda}$ operator:

$$
S = (1 - \varepsilon)\mathbf{\Lambda}S + \varepsilon B.
$$

If $\Lambda$ is expressed in matrix form (usually a terrible idea, but useful for demonstration purposes), we can solve the system using the inverse of a matrix:

$$
S = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda})^{-1}[\varepsilon B],
$$

The way to compute $\mathbf{\Lambda}S$ varies. We can use the Feautrier method to build the $\mathbf{\Lambda}$ matrix, which is doable for simple 1D problems for a single wavelength. However, when we solve the statistical equilibrium equations at the same time, using complete linearisation, for multiple wavelengths and transitions, the size of the $\mathbf{\Lambda}$ matrix becomes huge, and impractical to invert, which is very time-consuming operation. Even in 1D problems. Instead, we solve for $S$ using iterative methods. 

Computing $\mathbf{\Lambda}S$, when the $S$ is known, is actually not too difficult, and does not require building the full $\mathbf{\Lambda}$ matrix. For example, using the Feautrier method we can compute the $P$ arrays for every angle, and obtain $J$ by summing over them. For the two-level problem with coherent scattering, $S$ is obtained by iteratively solving the equation, using the classical $\Lambda$ iteration (CLI):

$$
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
$$
where $S^{(n)}$ is the estimate at iteration $n$. As an initial guess for $S^0$ different approaches are used, and $S^0=B$ works well except for the strongest lines. Since $\mathbf{\Lambda}[S^{(n)}]$ is not very expensive to calculate, one can simply iterate several times, until the fractional change $\left| (S^{(n+1)} - S^{(n)}) / S^{(n)} \right|$ is small enough. 

Unfortunately, the CLI does not work well in cases with $\varepsilon < 10^{-2}$. Even though the corrections to $S$ will decrease with the number of iterations, it will not converge to the correct result. Instead, a much better approach is to split the $\Lambda$ operator:

$$
\mathbf{\Lambda} = \mathbf{\Lambda} + (\mathbf{\Lambda} -\mathbf{\Lambda}^*),
$$

so that
$$
\begin{aligned}
J &= \mathbf{\Lambda}^*\left[ S \right] + (\mathbf{\Lambda} -\mathbf{\Lambda}^*)\left[ S \right],\\
S^{(n+1)} &= (1-\varepsilon)\mathbf{\Lambda^*}[S^{(n+1)}]  + (1-\varepsilon)(\mathbf{\Lambda} -\mathbf{\Lambda}^*)[S^{(n)}]+ \varepsilon B.
\end{aligned}
$$

This results in the approximate (or acelerated) $\Lambda$ iteration, which can be written as:

$$
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right],
$$

where 

$$
S^\mathrm{FS} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B.
$$

The real trick is knowing which approximate $\mathbf{\Lambda}^*$ operator to use. Some cases are listed in the lecture notes, but here we will use the OAB operator, which is simply the diagonal of the full $\mathbf{\Lambda}$ matrix. However, "simply the diagonal" can be deceptive. How does one obtain the diagonal of a matrix without building the matrix in the first place? We have seen above that 

$$
J = \sum_i\mathbf{T}^{-1}(\mu_i)S,
$$

so that

$$
\mathbf{\Lambda} = \sum_\mu\mathbf{T}^{-1}.
$$

Since the $\mathbf{T}$ is tridiagonal, we can use simple algorithms to compute the diagonal of its inverse, without the need to compute $\mathbf{\Lambda}$ explicitly. One such method was given by Usmani (1994), and is coded in the function below
.

In [2]:
@njit
def diag_inverse_tri(B):
    """
    Algorithm of Usmani 1994, Computers Math. Applic. 27, 8, 59-66.
    Does not seem to work for N > 1100. 

    The input is B, an array of shape (3, N), with the diagonal
    values, same syntax as scipy needed by scipy.linalg.solve_banded
    """
    n = B.shape[1]
    a = B[0]
    b = B[1]
    c = B[2]

    θ = numpy.zeros(n+2)
    ϕ = numpy.zeros(n+4)
    diag = numpy.zeros(n)

    θ[0] = 1
    θ[1] = b[0]
    for i in range(2, n+1):
        θ[i] = b[i-1] * θ[i-1] - a[i-1] * c[i-2] * θ[i-2]

    ϕ[n+1] = 1
    ϕ[n] = b[n-1]
    for i in range(n-1, 1, -1):
        ϕ[i] = b[i-1] * ϕ[i+1] - a[i] * c[i-1] * ϕ[i+2]

    for i in range(n):
        diag[i] = θ[i] * ϕ[i+2] / θ[n]

    return diag

Another alternative to obtain the diagonal $\mathbf{\Lambda}^*$ operator is to compute the diagonal at the same time as one solves the Feautrier via forward-backward substitution, which is implemented in the `feautrier` function above.

---

### Exercise 1: The Feautrier method and the $\mathbf{\Lambda}$ operator [30 points]

The Feautrier method consists of solving the equation

\begin{equation}
\mathbf{T}P = S,
\end{equation}

where $\mathbf{T}$ is a tridiagonal matrix. The function `Tmatrix` computes $\mathbf{T}$ for a single frequency and direction on a 1D plane-parallel atmosphere, given an optical depth scale and $\mu$. Below we use the C/Python convention that 0 is the first index of an array, and $n-1$ the last point.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[6 points]* At the top of the atmosphere, we can assume $I^-=0$. Why? Using a Taylor expansion around the Feautrier $P_0$ and assuming $I^-=0$ at the top of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the top can be written as:

$$
\begin{aligned}
B_0 &= \frac{2}{\Delta\tau^2} + \frac{2}{\Delta\tau} + 1, \\ 
C_0 &= \frac{2}{\Delta\tau^2}, \\
\Delta\tau &=\tau_{1} - \tau_{0}.
\end{aligned}
$$ 

* *[7 points]* At the bottom of the atmosphere, we can assume $I^+=S$. Why? Using a Taylor expansion around the Feautrier $P_{n-1}$, and assuming $I^+=S$ at the bottom of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the bottom can be written as:

$$
\begin{aligned}
A_{n-1} &= \frac{2}{\Delta\tau(\Delta\tau+2)},  \\ 
B_{n-1} &= \frac{2+2\Delta\tau +\Delta\tau^2}{\Delta\tau(\Delta\tau+2)},\\
\Delta\tau &=\tau_{n-1} - \tau_{n-2}.
\end{aligned}
$$

* *[8 points]* Using a Gaussian quadrature with 5 points, write a function called `solve_lambda_implicit()` to solve $J=\mathbf{\Lambda}S$ without explicitly building $\mathbf{\Lambda}$ as a matrix, taking as arguments $\tau$ and $S$. Do not use the `feautrier` function. Use it to solve the simple case where `tau = numpy.logspace(-4, 2, 50)` and $S=1$ at all depth points. Plot $J$ and $S$ vs $\log_{10}\tau$. Discuss the values of $J$ at $\tau\gg 1$ and $\tau\ll 1$. How do they compare with analytical solutions?

* *[9 points]* Now write a function called `solve_lambda_direct()` where you build the $\mathbf{\Lambda}$ matrix explicity and solve for $J$. Using `tau = numpy.logspace(-4, 2, 50)`, plot the $\mathbf{\Lambda}$ matrix and discuss its physical meaning. Explore the effect of using a different number of points in the Gaussian quadrature. What is a reasonable number of points?

</div>

### Exercise 2: $\Lambda$ iteration for coherent scattering in a two-level atom [35 points]

The source function for a two-level atom with coherent scattering can be written as

\begin{equation}
S = (1-\varepsilon)\mathbf{\Lambda}[S] + \varepsilon B,
\end{equation}
and a direct solution can be written as
\begin{equation}
S = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda})^{-1}[\varepsilon B],
\end{equation}

which involves inverting matrices (impractical for most real-life applications). Other numerical methods to solve for $S$ are the classical $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
\end{equation}
Where we need a guess for $S^{(1)}$, often $S^{(1)}=B.$ An alternative that is much more efficient is the approximate (or accelerated) $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right].
\end{equation}

For the iterative schemes, we can measure the convergence speed by calculating the fractional difference between source functions of successive iterations, e.g.:
\begin{equation}
\delta = \left|\left|\frac{S_\nu^{(n+1)}-S_\nu^{(n)}}{S_\nu^{(n)}}\right| \right|.
\end{equation}

Throughout this exercise, use a Gaussian quadrature with 5 points as your angle quadrature.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[10 points]* On a similar vein to what you did in the previous exercise, write a function called `solve_cs_direct()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using a direct solution for the problem of coherent scattering in a two-level atom (using matrix inversions). Use it to solve the problem for $\varepsilon=10^{-3}$, `tau = numpy.logspace(-4, 4, 50)` and $B=1.5\tau$. Do a log-log plot for S, J, and B. In this case, why is $J>B$ at the surface?
    
* *[11 points]* Write a function `solve_cs_CLI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the classical $\Lambda$ iteration (CLI). For the simple case where $B=1$ everywhere, $\varepsilon=10^{-2}$, plot $S$, $J$, $B$, for CLI and the direct solution from the previous question. Does CLI work well for this case? How many iterations do you need to achieve $\delta < 10^{-3}$?
    
* *[14 points]* Write a function `solve_cs_ALI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the accelerated $\Lambda$ iteration (ALI). For the $\mathbf{\Lambda}^*$ operator, use the OAB operator, which simply uses the diagonal of the full $\mathbf{\Lambda}$ operator. Usually, this $\mathbf{\Lambda}^*$ is computed without having to build the $\mathbf{\Lambda}$ explicitly, during the tridiagonal solution of $\mathbf{T}P = S$. Use the provided function `diag_inverse_tri()` to obtain $\mathbf{\Lambda}^*$ from the inverse of $\mathbf{T}$ matrices (as before, account for different angle quadratures), using `Tmatrix(tau, mu, format='banded')` to get $\mathbf{T}$ in the correct format. For the simple case where $\varepsilon=10^{-5}$, `tau = numpy.logspace(-4, 4, 50)`, `B = tau[::-1] * 1.5` , plot $S$ and $B$, for $S$ computed with the direct solution, CLI, and ALI, both with a maximum of 100 iterations. Discuss the differences. How many iterations of ALI do you need to be close to the direct solution? And how many iterations of CLI?
    
    
</div>

### Exercise 3: Two-level non-LTE problem from a 1D atmosphere [30 points]

Now we will relax some of the previous approximations and work on a more realistic case: solving the non-LTE problem for a spectral line from a two-level atom in a 1D plane-parallel atmosphere. To avoid excessive coding, we will not solve the statistical equilibrium equations and will compute the line in a much more approximated fashion that you did in Project B. 

We will no longer assume coherent scattering, but instead complete redistribution, so that

$$
\begin{aligned}
S &= (1-\varepsilon)\overline{J} + \varepsilon B, \\
\overline{J} &= \overline{\mathbf{\Lambda}}\left[S\right]. \\
\end{aligned}
$$

You will solve the problem using ALI with a diagonal operator and a Feautrier solver. Since we have several frequencies, $\overline{\mathbf{\Lambda}}^*$ is now averaged over angle and frequency:

\begin{equation}
\overline{\mathbf{\Lambda}}^* = \frac{1}{2} \int_{-1}^{1}\int_0 ^ \infty \varphi(\nu-\nu_0) \mathbf{\Lambda}^*_{\mu\nu} \mathrm{d}\nu \mathrm{d}\mu,
\end{equation}

where $\varphi(\nu-\nu_0)$ is the line profile, and $\mathbf{\Lambda}^*_{\mu\nu}$ the diagonal operator for a given frequency and direction.

You will make use of the provided function `feautrier()`, which computes the Feautrie solution as you did in previous questions, but in a form that is numerically more stable. For a given $\tau$ and $S$, it returns the Feautrier $P_\mu\nu$ (if `doLstar=True`) and also $I^+$ and $I^-$ (if `doI=True`). As before, use a Gaussian quadrature with 5 points as your angle quadrature. Note that `feautrier()` works on a given ray and does not take $\mu$ as an argument. For inclined rays in the 1D plane-parallel case, you need to pass it $\tau/\mu$, where $\tau$ is your vertical optical depth scale.

We create $\varphi(\nu)$ from a Voigt profile. To facilitate the frequency integration of $\overline{\mathbf{\Lambda}}^*$, when we obtain the profile we also compute integration weights $w_i$ so that the frequency integration is a simple weighed sum:

$$
\int_0^\infty \varphi(\nu-\nu_0)\mathrm{d}\nu = \sum_i w_i \varphi_i = 1.
$$
Instead of writing a detailed function to compute the line extinction $\alpha_\nu^l$, we will just create $\alpha_\nu^l$ by scaling $\alpha^c$ by the line profile and an $\eta$ factor.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[4 points]* Create a function called `profile()` that returns a Voigt profile $\varphi_\nu$ and integration weights $w$ as function of dimensionless frequency $u$. It should have as arguments `u_max` (the maximum range in dimensionless frequencies, which should go from `-u_max` to `+u_max`), the damping parameter $a$, and the number of points $n_\nu$. Create another function called `tau_scale()` that will calculate the optical depths scaled by the line profile and $\eta$, so that at the wings $\tau_0^l = \tau_c$ and at the line core, $\tau_{\nu 0}^l = \tau_c \eta$. It should take as arguments the continuum optical depth scale (a 1D array of depth points), the line profile (a 1D array of frequency points), and $\eta$, the ratio of line to continuum extinction. It should return a 2D array of $\tau_\nu^l$, where one dimension is depth and the second dimension is frequency.  
        
* *[12 points]* Write a non-LTE solver for the two-level case with complete redistribution, in a function called `solve_twolevel()`. As before, use ALI with the diagonal operator, in this case $\overline{\mathbf{\Lambda}}^*$. Make use of the `feautrier()` to obtain both $P_\mu\nu$ (and integrating over angle and frequency to obtain $\overline{J}$) and $L^*$, the diagonal of 
$\overline{\mathbf{\Lambda}}^*$. `solve_twolevel()` should take as arguments $B$ (1D array), $\varepsilon$ (1D array), $\varphi_\nu$ (1D array), and $\eta$ (scalar), and return S and $\overline{J}$. It should be used together with your previous functions `profile()` and `tau_scale()`.
    
* *[8 points]* Use `solve_twolevel()` for the FALC model atmosphere. Use the atmosphere's $\tau_{500}$ as $\tau_c$, and create a line profile with `u_max=5`, $a$=0.1, and 51 frequency points. The free parameters here will be $\varepsilon$ and $\eta$. Study the case where $\varepsilon$ is constant in the atmosphere. Adjust $\varepsilon$ and $\eta$ you obtain a line that thermalises in the photosphere ($\tau^c\approx 1$, line profile should be in absorption), and one that thermalises in the chromosphere ($\tau^c \approx 10^{-4}$, line profile should have emission in the core). Discuss. 
    
* *[6 points]* In real atmospheres, $\varepsilon$ varies with height. Assume $\eta=10^6$, compare a case with a constant $\varepsilon=10^{-3}$ with the case where $\varepsilon$ is given by `epsilon` below, which starts at 1 but then drops down to a minimum of $10^{-4}$. Why is $J$ at the surface higher in the constant $\varepsilon=10^{-3}$ , even though the second case has a lower value of $\varepsilon$ ($10^{-4}$) at the surface?

```python
tmp = numpy.linspace(-30, 10, len(falc['tau_500']))
epsilon = 1/(1 + numpy.exp(-(tmp) )) + 1e-4
```
</div>

***Hints:*** Do not use units with `feautrier`. You can obtain $B$ from FALC with units, but before passing any arrays to `feautrier` make sure they do not have units (e.g. take `.value` for $B$ and $\tau_{500}$).
    
